# 1.5 「ファインチューニング」で精度向上を実現する方法  
- 本ファイルでは、学習済みのVGGモデルを使用し、ファインチューニングでアリとハチの画像を分類するモデルを学習します

# 学習目標  
1.PyTorchでGPUを使用する実装コードを書けるようになる  
2.最適化手法の設定において、層ごとに異なる学習率を設定したファインチューニングを実装できるようになる  
3.学習したネットワークを保存・ロードできるようになる 

In [ ]:
# パッケージのimport
import os
import numpy as np
import glob
import tensorflow as tf
import json
from PIL import Image
import io
import matplotlib.pyplot as plt
%matplotlib inline
import random


from tqdm import tqdm

import torch
import torchvision
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torchvision import models, transforms
from sklearn.model_selection import train_test_split 


In [ ]:
# PyTorchのバージョン確認
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

In [ ]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

# 入力画像の前処理クラスを作成

In [ ]:

resize = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

train_transforms = transforms.Compose([
                transforms.RandomResizedCrop(
                    resize, scale=(0.5, 1.0)),  # データオーギュメンテーション
                transforms.RandomHorizontalFlip(),  # データオーギュメンテーション
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize(mean, std)  # 標準化
    ])
val_transforms = transforms.Compose([
                transforms.Resize(resize),  # リサイズ
                transforms.CenterCrop(resize),  # 画像中央をresize×resizeで切り取り
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize(mean, std)  # 標準化
    ])
test_transforms = transforms.Compose([
                transforms.Resize(resize),  # リサイズ
                transforms.CenterCrop(resize),  # 画像中央をresize×resizeで切り取り
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize(mean, std)  # 標準化
    ])

# テストデータを表示

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import os
import pandas as pd
from scipy import stats
import numpy as np
import glob
import tensorflow as tf
# import timm
import random
import time
import copy
from operator import itemgetter

from collections import OrderedDict, namedtuple
import joblib

import logging
import sys

import PIL
import cv2
import albumentations as A
import io
import IPython.display as display

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
import torch.optim as optim
# import torch_xla
# import torch_xla.core.xla_model as xm
# import torch_xla.debug.metrics as met
# import torch_xla.distributed.parallel_loader as pl
# import torch_xla.distributed.xla_multiprocessing as xmp
# import torch_xla.utils.utils as xu

import torchvision
# from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import torchvision.transforms as transforms
from torchvision.utils import make_grid

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics, model_selection

import warnings
warnings.filterwarnings("ignore");

train_files = glob.glob('../input/tpu-getting-started/*224/train/*.tfrec')
val_files = glob.glob('../input/tpu-getting-started/*224/val/*.tfrec')
test_files = glob.glob('../input/tpu-getting-started/*224/test/*.tfrec')

In [ ]:
def parse_tfrec_data(files, test=False):
    if not test: 
        feature_description = {
            'class': tf.io.FixedLenFeature([], tf.int64),
            'id': tf.io.FixedLenFeature([], tf.string),
            'image': tf.io.FixedLenFeature([], tf.string),
        }
    else:
        feature_description = {
        'id': tf.io.FixedLenFeature([], tf.string),
        'image': tf.io.FixedLenFeature([], tf.string),
    }
    parse_image_f = lambda x: tf.io.parse_single_example(x, feature_description)

    ids = []
    images = []
    if not test:
        cl = []

    for i in tqdm(files):
        image_dataset = tf.data.TFRecordDataset(i)
        image_dataset = image_dataset.map(parse_image_f)

        ids_ = [str(id_features['id'].numpy())[2:-1] for id_features in image_dataset] # [2:-1] is done to remove b' from 1st and 'from last in train id names
        ids = ids + ids_

        images_ = [image_features['image'].numpy() for image_features in image_dataset]
        images = images + images_

        if not test:
                cl_ = [int(class_features['class'].numpy()) for class_features in image_dataset]
                cl = cl + cl_
    if test:
        return ids, images
    else:
        return ids, cl, images

In [ ]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

train_ids, train_cl, train_images = parse_tfrec_data(train_files)
val_ids, val_cl, val_images = parse_tfrec_data(val_files)
test_ids, test_images = parse_tfrec_data(test_files, test=True)

In [ ]:
# 画像前処理の動作を確認

# 1. 画像読み込み
#image_file_path = '../input/tpu-getting-started/tfrecords-jpeg-224x224/test/00-224x224-462.tfrec'
#img = Image.open(image_file_path)  # [高さ][幅][色RGB]
print(train_ids[0])
print(train_cl[0])
img = Image.open(io.BytesIO(train_images[0]))
img

# 2. 元の画像の表示
plt.imshow(img)
plt.show()

# 3. 画像の前処理と処理済み画像の表示
img_transformed = train_transforms(img)  # torch.Size([3, 224, 224])

# (色、高さ、幅)を (高さ、幅、色)に変換し、0-1に値を制限して表示
img_transformed = img_transformed.numpy().transpose((1, 2, 0))
img_transformed = np.clip(img_transformed, 0, 1)
plt.imshow(img_transformed)
plt.show()

# DataSetを作成

In [ ]:
class FlowerDataset(data.Dataset):
    
    def __init__(self, ids, images, cl, transforms, test=False):
        
        self.ids = ids
        self.images = images
        if not test:
            self.cl = cl
        self.transforms = transforms
        self.is_test = test
    
    def __len__(self):
        return len(self.ids)
        
    def __getitem__(self, idx):
        
        img = self.images[idx]
        img = Image.open(io.BytesIO(img))
        #img = np.array(img)
        img = self.transforms(img)
        
        if self.is_test:
            return img, -1, self.ids[idx]
        return img, int(self.cl[idx]), self.ids[idx]

train_dataset = FlowerDataset(train_ids, train_images, train_cl, transforms=train_transforms)
val_dataset = FlowerDataset(val_ids, val_images, val_cl, transforms=val_transforms)
test_dataset = FlowerDataset(test_ids, test_images, None, transforms=test_transforms, test=True)

# DataLoaderを作成

In [ ]:
IMG_SIZE = 224

# DataLoaderを作成する
batch_size = 32

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader, "test": test_loader}

# ネットワークモデルの作成

In [ ]:

!pip install efficientnet_pytorch

In [ ]:
# 学習済みのVGG-16モデルをロード

# VGG-16モデルのインスタンスを生成
use_pretrained = True  # 学習済みのパラメータを使用
#use_pretrained = False  # 学習済みのパラメータを使用
#net = models.vgg16(pretrained=use_pretrained)


# VGG16の最後の出力層の出力ユニットを病気4種と健康の5つに付け替える
#net.classifier[6] = nn.Linear(in_features=4096, out_features=104)


from efficientnet_pytorch import EfficientNet

net = EfficientNet.from_pretrained('efficientnet-b4')#Pretrained_modelのインポート
#事前学習にはImageNetを用いているので、全結合層は1000クラスです。以下のようにクラス数を変更すれば転移学習に利用することができます。

num_ftrs = net._fc.in_features#全結合層の名前は"_fc"となっています
net._fc = nn.Linear(num_ftrs, 104)


# 訓練モードに設定
net.train()

print('ネットワーク設定完了：学習済みの重みをロードし、訓練モードに設定しました')

# 損失関数を定義
* 損失関数は、モデルが問題を学習出来ているかの指標”ロス(損失)”の計算方法を指定する

In [ ]:
# 損失関数の設定
criterion = nn.CrossEntropyLoss()

# 最適化手法を設定
* OptimizerがVGGモデルのどこのパラメータを更新(＝学習・最適化)するかを指定する
* Optimizerは”誤差逆伝搬法”を用いて、損失関数を最小化するようにVGGモデルのパラメータを更新(＝学習・最適化)役割を担う

In [ ]:
# ファインチューニングで学習させるパラメータを、変数params_to_updateの1～3に格納する
#
#params_to_update_1 = []
#params_to_update_2 = []
#params_to_update_3 = []

# 学習させる層のパラメータ名を指定
#update_param_names_1 = ["features"]
#update_param_names_2 = ["classifier.0.weight",
#                        "classifier.0.bias", "classifier.3.weight", "classifier.3.bias"]
#update_param_names_3 = ["classifier.6.weight", "classifier.6.bias"]

# パラメータごとに各リストに格納する
#for name, param in net.named_parameters():
#    if update_param_names_1[0] in name:
#        param.requires_grad = True
#        params_to_update_1.append(param)
#        print("params_to_update_1に格納：", name)
#
#    elif name in update_param_names_2:
#        param.requires_grad = True
#        params_to_update_2.append(param)
#        print("params_to_update_2に格納：", name)
#
#    elif name in update_param_names_3:
#        param.requires_grad = True
#        params_to_update_3.append(param)
#        print("params_to_update_3に格納：", name)
#
#    else:
#        param.requires_grad = False
#        print("勾配計算なし。学習しない：", name)

In [ ]:
# 最適化手法の設定
#optimizer = optim.SGD([
#    {'params': params_to_update_1, 'lr': 1e-4},
#    {'params': params_to_update_2, 'lr': 5e-4},
#    {'params': params_to_update_3, 'lr': 1e-3}
#], momentum=0.9)
optimizer = optim.Adam(net.parameters())

# 学習・検証を実施

In [ ]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # 初期設定
    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # epochのループ
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数

            # 未学習時の検証性能を確かめるため、epoch=0の訓練は省略
            #if (epoch == 0) and (phase == 'train'):
            #    continue

            # データローダーからミニバッチを取り出すループ
            for inputs, labels, _ in tqdm(dataloaders_dict[phase]):

                # GPUが使えるならGPUにデータを送る
                inputs = inputs.to(device)
                labels = labels.to(device)
                                
                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)  # 損失を計算
                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # 結果の計算
                    epoch_loss += loss.item() * inputs.size(0)  # lossの合計を更新
                    # 正解数の合計を更新
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率を表示
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            

In [ ]:
# 学習・検証を実行する
num_epochs=50
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

# 学習したネットワークを保存・ロード

In [ ]:
# PyTorchのネットワークパラメータの保存
save_path = './weights_fine_tuning.pth'
torch.save(net.state_dict(), save_path)

In [ ]:
#結果投稿用のCSV作成関数
def make_submission(model, test_loader, cuda=True):

    results=[]
    for batch, _, ids in tqdm(test_loader):
        with torch.no_grad():
            if cuda:
                batch = batch.cuda()
            model.eval()
            out = model(batch)
            pred_labels = torch.argmax(out.data.cpu(), dim=1)
            rows = list(
                zip(
                    list(ids), list(pred_labels.numpy().tolist())
                )
            )
            results.append(pd.DataFrame(rows, columns=['id', 'label']))
    result_df = pd.concat(results)
    result_df['label'] = result_df['label'].astype(int)
    return result_df


In [ ]:
# PyTorchのネットワークパラメータのロード
load_path = './weights_fine_tuning.pth'
load_weights = torch.load(load_path)
net.load_state_dict(load_weights)

# GPU上で保存された重みをCPU上でロードする場合
#load_weights = torch.load(load_path, map_location={'cuda:0': 'cpu'})
#net.load_state_dict(load_weights)

submission = make_submission(net, test_loader)
submission.to_csv('submission.csv', index=False)